# Segmenting and Clustering Neighborhoods in Toronto

In [ ]:
import pandas as pd
import requests
import lxml.html as lh
from urllib.request import urlopen
from bs4 import BeautifulSoup

Download the url content from wikipedia

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#page = requests.get(url)
#Store the contents of the website under doc
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')

Use BeautifulSoup library to get the table contents

In [27]:
import re
# Get rows by looking for tr
rows = soup.find_all('tr')

# Get cells
for row in rows:
    row_td = row.find_all('td')
    
str_cells = str(row_td)
cleantext = BeautifulSoup(str_cells, "lxml").get_text()

list_rows = []
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)

Load the table data into a pandas dataframe

In [28]:
# Create a data frame with the "list_rows"
df = pd.DataFrame(list_rows) 
# Expand in cells separated by commas
df = df[0].str.split(',', expand=True)
# Remove '[', ']' and '\n' characters
df[0] = df[0].str.strip('[')
df[2] = df[2].str.strip(']')
df[2] = df[2].str.replace('\n','')

Get Table Header by looking for the 'th' tag

In [29]:
# Get table header
col_labels = soup.find_all('th')
all_header = []
col_str = str(col_labels)
cleantext2 = BeautifulSoup(col_str, "lxml").get_text()
all_header.append(cleantext2)

In [30]:
# Add headers to a new dataframe
df2 = pd.DataFrame(all_header)
# Clean headers DF
df2 = df2[0].str.split(',', expand=True)
df2 = df2.drop([3],axis=1)
df2[0] = df2[0].str.strip('[')
df2[2] = df2[2].str.replace('\n','')

In [31]:
# Concatenate header df (df2) and contents df (df)
frames = [df2, df]
df = pd.concat(frames)
# set firs row as columns header
df = df.rename(columns=df.iloc[0])
# get only with the first three columns
df = df.iloc[:,0:3]
# remove empty spaces
df.columns = df.columns.str.lstrip()
# remove fist row
df=df.drop([0],axis=0)

Remove Not assigned rows

In [32]:
# Remove Not assigned rows
df=df[df.Borough.str.contains('Not assigned')==False]
# Remove empty rows
df=df[df.Postcode.str.contains('\n')==False]

Combine Neighbourhood columns for rows with same postcode

In [38]:
# Combine Neighbourhood columns for rows with same postcode
df=df.groupby('Postcode').agg({'Borough': 'first', 'Neighbourhood': lambda x: ','.join(x)})
df=df.reset_index()

Set neighborhood as borough for the "Not assigned" neighborhoods

In [39]:
# Set neighborhood as borough for the "Not assigned" neighborhoods
df.loc[df.Neighbourhood.str.contains('Not assigned'),'Neighbourhood']=df.loc[df.Neighbourhood.str.contains('Not assigned'),'Borough']

In [40]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [41]:
df.shape

(103, 3)